In [1]:
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import xlsxwriter
import time
import requests
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myApp")
from geopy.geocoders import ArcGIS
nom = ArcGIS()

In [2]:
titles = []
ubics = []
clase = []
links = []
ambs = []
dorms = []
url = 'https://inmuebles.mercadolibre.com.ar/alquiler-temporario/misiones/posadas/'

In [3]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}
page = requests.get(url, headers=headers)
soup1 = BeautifulSoup(page.content, "lxml")
soup2 = BeautifulSoup(soup1.prettify(), "lxml")

In [7]:
for x in soup2.find_all('li', attrs={'class':"ui-search-layout__item"}):
    title = x.find('h2', attrs={'class': "ui-search-item__title shops__item-title"})
    if title is None:
        titles.append(np.NaN)
    else:
        titles.append(title.text)

    ubic = x.find('span', attrs={'class': "ui-search-item__location-label"})
    if ubic is None:
        ubics.append(np.NaN)
    else:
        ubics.append(ubic.text)
        
    info = x.find('span', attrs={'class': "ui-search-item__group__element ui-search-item__subtitle-grid shops__items-group-details"})
    if info is None:
        clase.append(np.NaN)
    else:
        clase.append(info.text)
        
    link = x.find('a')["href"]
    links.append(link)


In [8]:
df = pd.DataFrame({'Alojamiento': titles, 'Tipo alojamiento':clase, 'Ubicación': ubics})        

In [9]:
#df.index = np.arange(1, len(df) + 1)
df = df.replace(r'\n',' ', regex=True)
df['Alojamiento'] = df['Alojamiento'].str.strip()
df['Tipo alojamiento'] = df['Tipo alojamiento'].str.strip()
df['Ubicación'] = df['Ubicación'].str.strip()
df['Coordenadas'] = df['Ubicación'].apply(nom.geocode)

In [10]:
df['Latitud'] = df['Coordenadas'].apply(lambda x: x.latitude if x != None else None)
df['Longitud'] = df['Coordenadas'].apply(lambda x: x.longitude if x != None else None)

In [11]:
df['Geolocalización'] = df[['Latitud', 'Longitud']].to_numpy().tolist()

In [12]:
del df['Coordenadas']

In [13]:
del df['Latitud']
del df['Longitud']

In [14]:
df['Sitio web'] = 'MercadoLibre'

In [15]:
df['Fecha consulta'] = pd.Timestamp.today().strftime('%d/%m/%Y')

In [16]:
df

,Alojamiento,Tipo alojamiento,Ubicación,Geolocalización,Sitio web,Fecha consulta
0,Espacio Alberdi,Casa en alquiler temporal,"Alberdi 965, Villa Sarita, Posadas...","[-27.359733460078, -55.893628430275]",MercadoLibre,08/09/2023
1,Moroty 1 - 2 - Monoambientes De Alquiler Tempo...,Departamento en alquiler temporal,"Perito Moreno 4127, Posadas, Misio...","[-27.387674368311, -55.902759941878]",MercadoLibre,08/09/2023
2,Departamento Por Día Posadas Misiones. Consult...,Departamento en alquiler temporal,"Mariano Moreno Sn, Otros Barrios,...","[-38.958840209347, -68.04239492425]",MercadoLibre,08/09/2023
3,Casa Para 10 Personas,Casa en alquiler temporal,"Avda Tierra Del Fuego 1829, Otros ...","[-54.0, -70.0]",MercadoLibre,08/09/2023
4,Departamento Céntrico Por Día,Departamento en alquiler temporal,"Jujuy 1845, Otros Barrios, Posadas...","[-27.367674011619, -55.900008933533]",MercadoLibre,08/09/2023
5,Excelente Departamento Temporario Pleno Centro...,Departamento en alquiler temporal,"Jujuy 1845, Posadas, Misiones, Arg...","[-27.367674011619, -55.900008933533]",MercadoLibre,08/09/2023
6,Casa En Alquiler Por Día,Casa en alquiler temporal,"25 De Mayo 2484, Otros Barrios, P...","[-25.10239, -65.53463]",MercadoLibre,08/09/2023
7,Departamento 2 Dormitorios En Posadas. Alquile...,Departamento en alquiler temporal,"3300, Posadas, Misiones","[-27.41879787, -55.963545482]",MercadoLibre,08/09/2023
8,Departamento En Alquiler Temporal En Posadas,Departamento en alquiler temporal,"124 Al 6300, Posadas, Misiones","[-27.399110104637, -55.941808109118]",MercadoLibre,08/09/2023
9,"Departamento Temporario Veraneo, Luminoso, Con...",Departamento en alquiler temporal,"Av. Tambor De Tacuarí, Posadas, Mi...","[-27.367603102745, -55.928628868142]",MercadoLibre,08/09/2023


In [17]:
df.to_excel('MercadoLibreAlqtemporal_Posadas_0923.xlsx', index=False)